# 高频考点：马尔科夫链分析

以下为四份考试卷中涉及马尔科夫链分析的题目详细讲解：

---

## 2024年8月考试

### Problem 3: Markov Chains

#### 考点:
1. 状态转移矩阵计算。
2. 判断马尔科夫链是否不可约。
3. 判断马尔科夫链是否非周期性并计算每个状态的周期。
4. 计算从状态A到状态B在5步内的转移概率。
5. 计算首次到达目标状态的概率分布。

#### 详细解析:
1. **状态转移矩阵计算**:
   - 题目要求明确转移矩阵的形式。应确保矩阵中的每一行的概率和为1。

2. **不可约性判断**:
   - 判断标准是是否存在从任意状态到任意其他状态的路径。
   - 使用连通性分析图（graph connectivity）可帮助直观判断。

3. **非周期性判断**:
   - 非周期性是指从任意状态返回自身的步数没有固定周期。
   - 计算每个状态的周期，并验证是否存在周期为1的状态。

4. **多步转移概率计算**:
   - 利用转移矩阵的幂运算（矩阵乘法）计算多步转移概率。例如，转移矩阵的平方代表两步转移的概率。

5. **首次到达目标状态的概率分布**:
   - 定义首次到达目标状态的时间T，利用递归方程计算每个可能的T值的概率。

---

## 2022年1月考试

### Problem 5: Markovian Travel

#### 考点:
1. 加载旅行数据。
2. 推导状态转移矩阵。
3. 计算平稳分布。
4. 从指定起点出发，计算3步后返回初始状态的概率。

#### 详细解析:
1. **加载旅行数据**:
   - 从文件中读取数据，将城市转换为编号表示（如用整数索引每个城市）。

2. **状态转移矩阵计算**:
   - 从数据中统计每个状态转移的频率，归一化以获得概率矩阵。

3. **平稳分布计算**:
   - 解状态转移矩阵的特征值问题，找到主特征值为1的对应特征向量并归一化。

4. **多步回归概率**:
   - 使用矩阵幂计算指定步数的转移概率。

---

## 2023年1月考试

### Problem 1: Markov Chain Analysis

#### 考点:
1. 从郊区出发，计算两步后到达市中心的概率。
2. 计算首次在两步后到达市中心的概率。
3. 判断马尔科夫链是否不可约。
4. 计算平稳分布。
5. 计算从市中心出发首次到达郊区的期望步数。

#### 详细解析:
1. **两步后到达市中心的概率**:
   - 使用转移矩阵计算。具体公式为 \(P(X_2 = “ downtown” | X_0 = “ suburb”)\)。

2. **首次两步后到达市中心的概率**:
   - 需要排除在第一步已到达市中心的概率，再计算两步的转移概率。

3. **不可约性判断**:
   - 验证是否每个状态都可以通过有限步数到达其他状态。

4. **平稳分布**:
   - 解方程 \(\pi P = \pi\)，其中\(\pi\)为平稳分布。

5. **击中时间期望**:
   - 定义击中时间随机变量T，递归计算期望值。

---

## 2024年1月考试

### Problem 3: Markov Chains

#### 考点:
1. 给出马尔科夫链的转移矩阵。
2. 判断马尔科夫链是否不可约。
3. 判断马尔科夫链是否非周期性并计算状态周期。
4. 确定马尔科夫链是否具有平稳分布并计算分布值。
5. 判断马尔科夫链是否可逆。

#### 详细解析:
1. **转移矩阵**:
   - 每个状态的出边概率之和需为1。

2. **不可约性**:
   - 验证每对状态是否连通。

3. **非周期性与周期**:
   - 计算所有状态的周期，检查最小公倍数是否为1。

4. **平稳分布**:
   - 如果存在平稳分布，解特征值为1的特征方程。

5. **可逆性**:
   - 验证详细平衡条件 \(\pi_iP_{ij} = \pi_jP_{ji}\)。

In [3]:
import numpy as np
import networkx as nx
from functools import reduce
from math import gcd
from scipy.linalg import eig

## 2024年8月考试

### Problem 3: Markov Chains

#### 考点:
1. 状态转移矩阵计算。

In [54]:
problem3_A = np.array([
    [0, 0.2, 0, 0.8],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [0.5, 0, 0.5, 0]
])

problem3_B = np.array([
    [0, 1, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0],
    [0, 0.5, 0, 0.5, 0, 0],
    [0, 0, 0.5, 0, 0.5, 0],
    [0, 0, 0, 0, 0, 1],
    [0.5, 0, 0, 0, 0.5, 0]
])

Ps=[problem3_A,problem3_B]
# 验证转移矩阵的合法性
def is_valid_transition_matrix(P):
    # 1. 检查是否为方阵
    if P.shape[0] != P.shape[1]:
        return False, "The matrix is not square."

    # 2. 检查是否所有元素为非负数
    if not np.all(P >= 0):
        return False, "The matrix contains negative elements."

    # 3. 检查每行是否归一化为 1
    if not np.allclose(np.sum(P, axis=1), 1):
        return False, "The rows do not sum to 1."

    return True, "The matrix is a valid transition matrix."

# 验证转移矩阵
is_vlid_list=[]
for i, p in enumerate(Ps):  # 使用 enumerate 获取索引 i 和矩阵 p
    is_valid, message = is_valid_transition_matrix(p)
    is_vlid_list.append(is_valid)
    
    # 输出验证结果
    print(f"Markov chain and Digraph have been successfully constructed.")
    print(f"The size of transition matrix P_{i} is: {p.shape}")
    print(f"Whether it is a qualified transition matrix: {message}")

Markov chain and Digraph have been successfully constructed.
The size of transition matrix P_0 is: (4, 4)
Whether it is a qualified transition matrix: The matrix is a valid transition matrix.
Markov chain and Digraph have been successfully constructed.
The size of transition matrix P_1 is: (6, 6)
Whether it is a qualified transition matrix: The matrix is a valid transition matrix.


2. **不可约性判断**:
   - 判断标准是是否存在从任意状态到任意其他状态的路径。(从任意一个状态出发，都能在有限步数内到达任意其他状态)
   - 如果马尔可夫链不是不可约的，则它可以划分为多个互相独立的子链（称为通信类）。
   - 对于不可约的马尔可夫链，通常可以找到一个唯一的平稳分布。
   - 如果链是可约的，每个独立子链可能有自己的平稳分布。
   - 使用连通性分析图（graph connectivity）可帮助直观判断。

In [60]:
# 如果转移矩阵有效，构建有向图
G_list=[]
for i, p in enumerate(Ps):
    if is_vlid_list[i]:
        # 1. 构建有向图
        G = nx.DiGraph()
        
        # 添加边和权重
        for i in range(p.shape[0]):
            for j in range(p.shape[1]):
                if p[i, j] > 0:  # 仅添加非零权重的边
                    G.add_edge(i, j, weight=p[i, j])
        
        # 打印构建结果
        print("The directed graph (Digraph) has been successfully constructed.")
        print("Number of nodes:", G.number_of_nodes())
        print("Number of edges:", G.number_of_edges())
    
        # 可视化或分析
        print("Graph edges with weights:")
        for u, v, data in G.edges(data=True):
            print(f"Edge from {u} to {v}, weight = {data['weight']}")
        G_list.append(G)
    else:
        print("The transition matrix is not valid. Please check your input.")
        G_list.append(None)
        
for G in G_list:
    is_irreducible = nx.is_strongly_connected(G)
    print("Is the Markov chain irreducible?", is_irreducible)

The directed graph (Digraph) has been successfully constructed.
Number of nodes: 4
Number of edges: 6
Graph edges with weights:
Edge from 0 to 1, weight = 0.2
Edge from 0 to 3, weight = 0.8
Edge from 1 to 2, weight = 1.0
Edge from 3 to 0, weight = 0.5
Edge from 3 to 2, weight = 0.5
Edge from 2 to 1, weight = 1.0
The directed graph (Digraph) has been successfully constructed.
Number of nodes: 6
Number of edges: 9
Graph edges with weights:
Edge from 0 to 1, weight = 1.0
Edge from 1 to 2, weight = 1.0
Edge from 2 to 1, weight = 0.5
Edge from 2 to 3, weight = 0.5
Edge from 3 to 2, weight = 0.5
Edge from 3 to 4, weight = 0.5
Edge from 4 to 5, weight = 1.0
Edge from 5 to 0, weight = 0.5
Edge from 5 to 4, weight = 0.5
Is the Markov chain irreducible? False
Is the Markov chain irreducible? True


3. **非周期性（aperiodic）判断**:
   - 非周期性是指从任意状态返回自身的步数没有固定周期。
   - 马尔可夫链的非周期性是指链中每个状态的返回周期不是固定的，也就是说，从一个状态 i 出发返回到 i 的步数没有一个大于 1 的固定周期。
   - 周期性状态：
	•	如果从状态 i 出发，每隔固定的 d > 1 步必然返回该状态，则状态 i 是周期性的。
	•	例如：从 i 出发总是第 2 步、4 步、6 步……返回，这种状态的周期为 d = 2。
    - 非周期性状态：
	•	如果从状态 i 出发，可能在不同的步数返回（如 1 步、2 步、3 步等都可能返回），且没有一个固定的大于 1 的周期约束，则状态是非周期性的。
   - 如果状态的周期 d(i) = 1，则称状态 i 是 非周期性的。如果链中所有状态都是非周期性的，则整个马尔可夫链称为 非周期性链。
   - 计算每个状态的周期，并验证是否存在周期为1的状态。

In [70]:
#period
# 3. 计算状态的周期性 和 周期

# 计算状态 x 的返回时间集 T(x)
def get_return_times(P, x):
    n = len(P)
    times = []
    for t in range(1, n+1):
        if np.linalg.matrix_power(P, t)[x, x] > 0:
            times.append(t)
    return times

# 计算状态的周期
for P in Ps:
    periods = {}
    for x in range(len(P)):
        T_x = get_return_times(P, x)
        period = reduce(gcd, T_x)
        periods[x] = period
    
    # 输出结果
    for state, period in periods.items():
        print(f"State {state} has period: {period}")
        if period == 1:
            print(f"State {state} is aperiodic")
        else:
            print(f"State {state} is not aperiodic")
    
    # 判断整个马尔可夫链是否非周期性
    is_aperiodic = all(period == 1 for period in periods.values())
    if is_aperiodic:
        print("The Markov chain is aperiodic")
    else:
        print("The Markov chain is periodic")
    print('\n')

    
print("the period of A:",np.power(problem3_A,5),np.power(problem3_A,5)[0][1])
print('\n')
print("the period of B:",np.power(problem3_B,5),np.power(problem3_B,5)[0][1])

State 0 has period: 2
State 0 is not aperiodic
State 1 has period: 2
State 1 is not aperiodic
State 2 has period: 2
State 2 is not aperiodic
State 3 has period: 2
State 3 is not aperiodic
The Markov chain is periodic


State 0 has period: 6
State 0 is not aperiodic
State 1 has period: 2
State 1 is not aperiodic
State 2 has period: 2
State 2 is not aperiodic
State 3 has period: 2
State 3 is not aperiodic
State 4 has period: 2
State 4 is not aperiodic
State 5 has period: 2
State 5 is not aperiodic
The Markov chain is periodic


the period of A: [[0.0000e+00 3.2000e-04 0.0000e+00 3.2768e-01]
 [0.0000e+00 0.0000e+00 1.0000e+00 0.0000e+00]
 [0.0000e+00 1.0000e+00 0.0000e+00 0.0000e+00]
 [3.1250e-02 0.0000e+00 3.1250e-02 0.0000e+00]] 0.0003200000000000001


the period of B: [[0.      1.      0.      0.      0.      0.     ]
 [0.      0.      1.      0.      0.      0.     ]
 [0.      0.03125 0.      0.03125 0.      0.     ]
 [0.      0.      0.03125 0.      0.03125 0.     ]
 [0.      0.     

4. **多步转移概率计算**:
   - 马尔可夫链的多步转移概率是指从一个状态 i 到另一个状态 j 经过多步转移的概率。这是马尔可夫链的一个重要概念，用于描述链的长期行为或多步转移的可能性。
   - 一步转移概率：
   - 当 n = 1 时，P^1_{ij} 就是马尔可夫链的转移矩阵 P 的元素 P_{ij}。
   - 多步转移概率：
   - 对于 n > 1，可以通过转移矩阵的 n 次幂计算多步转移概率：
        P^n = P \cdot P \cdot P \cdots P \quad (n \text{ 次矩阵乘法})

	•	P^n_{ij} 是矩阵 P^n 的第 i 行第 j 列的元素。
	- 马尔可夫性质：
	•	多步转移概率满足马尔可夫链的性质：当前状态的分布只取决于上一状态，而与之前的状态无关。
   - 利用转移矩阵的幂运算（矩阵乘法）计算多步转移概率。例如，转移矩阵的平方代表两步转移的概率。

## 2024年1月考试

### Problem 3: Markov Chains

1. 给出马尔科夫链的转移矩阵。

In [107]:
P_A = np.array([
    [0.8, 0.2, 0, 0],
    [0.6, 0.2, 0.2, 0],
    [0, 0.4, 0, 0.6],
    [0, 0, 0.8, 0.2]
])

P_B = np.array([
    [0, 0.2, 0, 0.8],
    [0, 0, 1, 0],
    [0, 1, 0, 0],
    [0.5, 0, 0.5, 0]
])

P_C = np.array([
    [0.2, 0.3, 0, 0, 0.5],
    [0.2, 0.2, 0.6, 0, 0],
    [0, 0.4, 0, 0.6, 0],
    [0, 0, 0, 0.6, 0.4],
    [0, 0, 0, 0.4, 0.6]
])

P_D = np.array([
    [0.8, 0.2, 0, 0],
    [0.6, 0.2, 0.2, 0],
    [0, 0.4, 0, 0.6],
    [0.1, 0, 0.7, 0.2]
])

Ps = [P_A,P_B,P_C,P_D]

# 验证转移矩阵的合法性
def is_valid_transition_matrix(P):
    # 1. 检查是否为方阵
    if P.shape[0] != P.shape[1]:
        return False, "The matrix is not square."

    # 2. 检查是否所有元素为非负数
    if not np.all(P >= 0):
        return False, "The matrix contains negative elements."

    # 3. 检查每行是否归一化为 1
    if not np.allclose(np.sum(P, axis=1), 1):
        return False, "The rows do not sum to 1."

    return True, "The matrix is a valid transition matrix."

# 验证转移矩阵
is_vlid_list=[]
for i, p in enumerate(Ps):  # 使用 enumerate 获取索引 i 和矩阵 p
    is_valid, message = is_valid_transition_matrix(p)
    is_vlid_list.append(is_valid)
    
    # 输出验证结果
    print(f"Markov chain and Digraph have been successfully constructed.")
    print(f"The size of transition matrix P_{i} is: {p.shape}")
    print(f"Whether it is a qualified transition matrix: {message}")

Markov chain and Digraph have been successfully constructed.
The size of transition matrix P_0 is: (4, 4)
Whether it is a qualified transition matrix: The matrix is a valid transition matrix.
Markov chain and Digraph have been successfully constructed.
The size of transition matrix P_1 is: (4, 4)
Whether it is a qualified transition matrix: The matrix is a valid transition matrix.
Markov chain and Digraph have been successfully constructed.
The size of transition matrix P_2 is: (5, 5)
Whether it is a qualified transition matrix: The matrix is a valid transition matrix.
Markov chain and Digraph have been successfully constructed.
The size of transition matrix P_3 is: (4, 4)
Whether it is a qualified transition matrix: The matrix is a valid transition matrix.


2. 判断马尔科夫链是否不可约。

In [108]:
# 如果转移矩阵有效，构建有向图
G_list=[]
for i, p in enumerate(Ps):
    if is_vlid_list[i]:
        # 1. 构建有向图
        G = nx.DiGraph()
        
        # 添加边和权重
        for i in range(p.shape[0]):
            for j in range(p.shape[1]):
                if p[i, j] > 0:  # 仅添加非零权重的边
                    G.add_edge(i, j, weight=p[i, j])
        
        # 打印构建结果
        print("The directed graph (Digraph) has been successfully constructed.")
        print("Number of nodes:", G.number_of_nodes())
        print("Number of edges:", G.number_of_edges())
    
        # 可视化或分析
        print("Graph edges with weights:")
        for u, v, data in G.edges(data=True):
            print(f"Edge from {u} to {v}, weight = {data['weight']}")
        G_list.append(G)
    else:
        print("The transition matrix is not valid. Please check your input.")
        G_list.append(None)
        
for G in G_list:
    is_irreducible = nx.is_strongly_connected(G)
    print("Is the Markov chain irreducible?", is_irreducible)

The directed graph (Digraph) has been successfully constructed.
Number of nodes: 4
Number of edges: 9
Graph edges with weights:
Edge from 0 to 0, weight = 0.8
Edge from 0 to 1, weight = 0.2
Edge from 1 to 0, weight = 0.6
Edge from 1 to 1, weight = 0.2
Edge from 1 to 2, weight = 0.2
Edge from 2 to 1, weight = 0.4
Edge from 2 to 3, weight = 0.6
Edge from 3 to 2, weight = 0.8
Edge from 3 to 3, weight = 0.2
The directed graph (Digraph) has been successfully constructed.
Number of nodes: 4
Number of edges: 6
Graph edges with weights:
Edge from 0 to 1, weight = 0.2
Edge from 0 to 3, weight = 0.8
Edge from 1 to 2, weight = 1.0
Edge from 3 to 0, weight = 0.5
Edge from 3 to 2, weight = 0.5
Edge from 2 to 1, weight = 1.0
The directed graph (Digraph) has been successfully constructed.
Number of nodes: 5
Number of edges: 12
Graph edges with weights:
Edge from 0 to 0, weight = 0.2
Edge from 0 to 1, weight = 0.3
Edge from 0 to 4, weight = 0.5
Edge from 1 to 0, weight = 0.2
Edge from 1 to 1, weight =

3. 判断马尔科夫链是否非周期性并计算状态周期。

In [109]:
# 计算状态 x 的返回时间集 T(x)
def get_return_times(P, x):
    n = len(P)
    times = []
    for t in range(1, n+1):  # 计算到 n 步
        if np.linalg.matrix_power(P, t)[x, x] > 0:
            times.append(t)
    return times

# 计算状态的周期
for P in Ps:
    periods = {}
    for x in range(len(P)):
        T_x = get_return_times(P, x)
        if not T_x:  # 如果返回时间集为空
            periods[x] = -1  # 用 -1 表示无法返回自身
        else:
            period = reduce(gcd, T_x)
            periods[x] = period
    
    # 输出结果
    for state, period in periods.items():
        if period == -1:
            print(f"State {state} cannot return to itself")
        else:
            print(f"State {state} has period: {period}")
            if period == 1:
                print(f"State {state} is aperiodic")
            else:
                print(f"State {state} is not aperiodic")
    
    # 判断整个马尔可夫链是否非周期性
    is_aperiodic = all(period == 1 for period in periods.values() if period != -1)
    if is_aperiodic:
        print("The Markov chain is aperiodic")
    else:
        print("The Markov chain is periodic")
    print('\n')

State 0 has period: 1
State 0 is aperiodic
State 1 has period: 1
State 1 is aperiodic
State 2 has period: 1
State 2 is aperiodic
State 3 has period: 1
State 3 is aperiodic
The Markov chain is aperiodic


State 0 has period: 2
State 0 is not aperiodic
State 1 has period: 2
State 1 is not aperiodic
State 2 has period: 2
State 2 is not aperiodic
State 3 has period: 2
State 3 is not aperiodic
The Markov chain is periodic


State 0 has period: 1
State 0 is aperiodic
State 1 has period: 1
State 1 is aperiodic
State 2 has period: 1
State 2 is aperiodic
State 3 has period: 1
State 3 is aperiodic
State 4 has period: 1
State 4 is aperiodic
The Markov chain is aperiodic


State 0 has period: 1
State 0 is aperiodic
State 1 has period: 1
State 1 is aperiodic
State 2 has period: 1
State 2 is aperiodic
State 3 has period: 1
State 3 is aperiodic
The Markov chain is aperiodic




In [110]:
# 计算状态 x 的返回时间集 T(x)
def get_return_times(P, x, max_steps=100):
    n = len(P)
    times = []
    for t in range(1, max_steps + 1):  # 计算到 max_steps
        if np.linalg.matrix_power(P, t)[x, x] > 0:  # 检查是否可以返回自身
            times.append(t)
    return times

# 计算每个状态的周期，返回 numpy 数组
def calculate_periods(Ps):
    all_periods = []  # 存储所有马尔可夫链的周期结果
    for P in Ps:
        n_states = len(P)
        periods = np.zeros(n_states, dtype=int)  # 初始化周期数组
        for x in range(n_states):
            T_x = get_return_times(P, x, max_steps=100)  # 获取返回时间集
            if not T_x:  # 如果返回时间集为空
                periods[x] = -1  # 用 -1 表示无法返回自身
            else:
                periods[x] = reduce(gcd, T_x)  # 计算最大公约数
        all_periods.append(periods)
    return np.array(all_periods)  # 转为 numpy 数组


# 计算所有马尔可夫链的状态周期
#运行代码后将输出一个 NumPy 数组，其中每一行对应一个转移矩阵，每列对应该马尔可夫链的某个状态的周期
#值为 -1 表示状态无法返回自身。
all_periods = calculate_periods(Ps)
print(all_periods)

[array([1, 1, 1, 1]) array([2, 2, 2, 2]) array([1, 1, 1, 1, 1])
 array([1, 1, 1, 1])]


/var/folders/zk/tl438cqd7g92whd9q4d9c0xm0000gn/T/ipykernel_95556/580826165.py:23: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(all_periods)  # 转为 numpy 数组


4. 确定马尔科夫链是否具有平稳分布并计算分布值。

In [111]:
#stationary distribution
# 4. 平稳分布 (stationary distribution)
stt_st=[]
for P in Ps:
    w, v = eig(P, left=True, right=False)
    stationary = np.real(v[:, np.isclose(w, 1)])
    stationary = stationary / stationary.sum()
    stt_st.append(stationary)
    print("Stationary distribution:", stationary.ravel())
    print('\n')

Stationary distribution: [0.61538462 0.20512821 0.1025641  0.07692308]


Stationary distribution: [0.  0.5 0.5 0. ]


Stationary distribution: [0.  0.  0.  0.5 0.5]


Stationary distribution: [0.64516129 0.20430108 0.08602151 0.06451613]




5. 判断马尔科夫链是否可逆。

In [112]:
#reversible
for stationary in stt_st:
    # 计算平稳分布 π
    stationary = stationary.ravel()  # 将平稳分布从二维转换为一维
    is_reversible = True  # 初始假设是可逆的
    
    # 遍历所有状态对 (i, j)
    for i in range(len(P)):
        for j in range(len(P)):
            if not np.isclose(stationary[i] * P[i, j], stationary[j] * P[j, i]):
                is_reversible = False
                break
        if not is_reversible:
            break
    
    print("Is the Markov chain reversible?", is_reversible)

Is the Markov chain reversible? False
Is the Markov chain reversible? False
Is the Markov chain reversible? False
Is the Markov chain reversible? False


## 2023年1月考试

### Problem 1: Markov Chain Analysis